In [1]:
# Import PyTorch Data Loader Library
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchvision.models.segmentation import fcn_resnet50

# Other Library Imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Custom Imports
from DRAC_Dataloading import DRAC_Loader
from DRAC_Models import ResNetBinary

# Import Warnings to Ignore Silly Warnings!
import warnings
warnings.filterwarnings("ignore")

In [2]:
torch.__version__

'2.0.1'

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.get_device_name(0))

cuda:0
NVIDIA RTX A5000


In [4]:
## Default Variables ##
# Datasets #
#train_count = 50
#test_count = 10
#transforms = True

# Dataloader #
batch_size = 1

# Model #
model = fcn_resnet50(num_classes=3)
model_name = 'fcn_resnet50'
criterion = nn.CrossEntropyLoss(reduction='none')
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 200

In [5]:
# Key Data Locations
# Select 'train' or 'test' data.
train_data = DRAC_Loader(data_type = 'train', transform = None)
test_data = DRAC_Loader(data_type = 'test', transform = None)

In [6]:
# Load Items into data loaders
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [7]:
# Load first item in train_loader
for i, data in enumerate(train_loader):
    image, classes = data
    
    # Take the first item in classes
    classy = classes[0]
    
    # Print the class shape
    print(classy.shape)
    print(image.shape)
    
    # Permute the class
    image = image.permute(0,3,1,2)
    print(image.shape)
    
    # Break the loop
    break

torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024, 3])
torch.Size([1, 3, 1024, 1024])


In [8]:
# Training loop
model.to(device)
model.train()

for epoch in range(epochs):
    running_loss = 0.0
    
    for images, masks in train_loader:
        # Reorganize the images to be of shape [batch_size, channels, height, width]
        images = images.permute(0, 3, 1, 2)
        images = images.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)['out']
        
        # Initialize loss
        loss = 0
        
        for mask in masks:
            loss += criterion(outputs, mask.long())
        
        # Average the loss by the number of masks
        loss /= len(masks)
        
        # Take the average of the loss
        # loss = torch.mean(loss)
        
        # Backward pass and optimize
        print(loss.shape)
        loss.backward()
        optimizer.step()
        
        print(loss)
        
        running_loss += loss.item()
    
    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch {epoch+1}, Loss: {epoch_loss}')

torch.Size([1, 1024, 1024])


RuntimeError: grad can be implicitly created only for scalar outputs